To run this example locally, execute: `ploomber examples -n templates/ml-online`.

To start a free, hosted JupyterLab: [![binder-logo](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ploomber/binder-env/main?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fploomber%252Fprojects%26urlpath%3Dlab%252Ftree%252Fprojects%252Ftemplates/ml-online%252FREADME.ipynb%26branch%3Dmaster)

Found an issue? [Let us know.](https://github.com/ploomber/projects/issues/new?title=templates/ml-online%20issue)

Have questions? [Ask us anything on Slack.](http://community.ploomber.io/)



# Machine Learning pipeline with online API

<!-- start description -->
Load data, generate features, train a model, and deploy model with flask.
<!-- end description -->

Note: all commands must be executed in the `ml-online/` directory.

## Setup

```sh
pip install --editable ".[dev]"
```

## File layout

`src/ml_online`:

1. `pipeline-features.yaml`: feature engineering YAML spec
2. `pipeline.yaml`: training pipeline
3. `infer.py`: converts training pipeline to an inference pipeline
4. `service.py`: uses inference pipeline to serve predictions using Flask

## Training pipeline

In [1]:
%%sh
ploomber build

name        Ran?      Elapsed (s)    Percentage
----------  ------  -------------  ------------
get         True         0.017032      0.295349
sepal-area  True         0.049882      0.864997
petal-area  True         0.02654       0.460226
features    True         0.036309      0.629629
fit         True         5.63697      97.7498


Building task 'fit': 100%|██████████| 5/5 [00:23<00:00,  4.80s/it]


Output from the training pipeline saved in the `products/` folder.

## Online API

Copy the trained model inside the project's package:

In [2]:
%%sh
cp products/model.pickle src/ml_online/model.pickle

Start web application:

In [3]:
from os import environ
import subprocess

def start_flask():
    """Start Flask and wait until it's ready
    """
    proc = subprocess.Popen(['flask', 'run'],
                        env=dict(environ, FLASK_APP='ml_online.service'),
                        stdout=subprocess.PIPE,
                        stderr=subprocess.STDOUT)
    
    while True:
        out = proc.stdout.readline()
        print(out.decode().strip())
    
        if b'5000' in out:
            break
    
    return proc

In [4]:
proc = start_flask()

* Serving Flask app 'ml_online.service' (lazy loading)
* Environment: production
Use a production WSGI server instead.
* Debug mode: off


/Users/Edu/dev/ploomber/src/ploomber/spec/dagspec.py:336: UserWarning: The following placeholders are declared in the environment but unused in the spec: {'sample'}
warnings.warn('The following placeholders are declared in the '


100%|██████████| 5/5 [00:00<00:00, 3052.18it/s]
* Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


*Note:* `start_flask()` is the same as executing the following a terminal:

```sh
export FLASK_APP=ml_online.service
flask run
```

Open a new terminal to make predictions:

In [5]:
import requests

In [6]:
def make_request(data):
    """Hit the prediction endpoint
    """
    response = requests.post('http://127.0.0.1:5000/',
                             data=data,
                             headers={'Content-Type': 'application/json'})
    return response.json()

In [7]:
make_request('{"sepal length (cm)": 5.1, "sepal width (cm)": 3.5, "petal length (cm)": 1.4, "petal width (cm)": 0.2}')

{'prediction': 0}

*Note: The previous command is equivalent to running the following on the terminal*

```sh
curl -d  '{"sepal length (cm)": 5.1, "sepal width (cm)": 3.5, "petal length (cm)": 1.4, "petal width (cm)": 0.2}' -H 'Content-Type: application/json' http://127.0.0.1:5000/
```

In [8]:
make_request('{"sepal length (cm)": 5.9, "sepal width (cm)": 3.0, "petal length (cm)": 5.1, "petal width (cm)": 1.8}')

{'prediction': 2}

*Note: The previous command is equivalent to running the following on the terminal*

```sh
curl -d  '{"sepal length (cm)": 5.9, "sepal width (cm)": 3.0, "petal length (cm)": 5.1, "petal width (cm)": 1.8}' -H 'Content-Type: application/json' http://127.0.0.1:5000/
```

Note: Ploomber exports a Python object that encapsulates the entire inference pipeline (pre-processing + feature engineering + model inference). You can deploy it with any framework.

In [9]:
# terminate flask app
proc.kill()

## Testing

The example contains some basic unit tests. To run them:

In [10]:
%%sh
pytest -p no:warnings

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /Users/Edu/dev/projects-ploomber/templates/ml-online
plugins: Faker-9.2.0, anyio-3.3.3
collected 5 items

tests/test_infer.py ..                                                   [ 40%]
tests/test_service.py .                                                  [ 60%]
tests/test_train.py .                                                    [ 80%]
tests/test_wheel.py .                                                    [100%]

============================== 5 passed in 12.41s ==============================



## Packaging

This project is a Python package. You can generate a distribution archive (`tar.gz`) or a built distribution (`.whl`) for deployment:


```sh
python -m build
```


The previous command creates a `.whl` and a `.tar.gz` file in the `dist/` directory; both contain all the necessary pieces to serve predictions: dependencies, pre-processing code, and model file.